
# SAE Comparison Experiment - Google Colab
### This notebook runs your SAE comparison experiment with the existing code.

### 1. Setup Environment

In [14]:
# %%
# Check GPU
!nvidia-smi

# %%
# Clone your repository (replace with your actual repo URL)
!git clone https://github.com/StrikerAI-Innovation/mechanistic-interpretability-scale.git repo
%cd repo

# %%
# Install required packages
!pip install torch transformers datasets accelerate -q
!pip install numpy pandas matplotlib seaborn tqdm pyyaml -q
!pip install einops wandb -q

/bin/bash: line 1: nvidia-smi: command not found
Cloning into 'repo'...
remote: Enumerating objects: 37, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (30/30), done.
remote: Total 37 (delta 3), reused 25 (delta 1), pack-reused 0 (from 0)
Receiving objects: 100% (37/37), 36.94 KiB | 945.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/repo/repo/repo


### 2. Import and Test

In [16]:
# %%
import sys
import os
sys.path.append(os.getcwd())

# Test imports
try:
    from src.models.base_sae import BaseSAE
    from src.models.k_sparse_sae import KSparseSAE
    from src.models.hybrid_sae import HybridSAE
    from src.models.vanilla_sae import VanillaSAE
    from src.training.trainer import SAETrainer
    from src.utils.data_loading import create_dataloader
    from src.utils.model_loading import load_model
    print("✓ All imports successful!")
except Exception as e:
    print(f"❌ Import error: {e}")

# %%
# Check PyTorch and GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

❌ Import error: No module named 'src.models.gated_sae'
PyTorch version: 2.6.0+cu124
CUDA available: False


### 3. Create Simplified Config

**bold text**# %%
# Create a Colab-friendly config
import yaml

colab_config = {
    'seed': 42,
    'dataset': 'wikitext',
    'max_train_samples': 5000,   # Reduced for Colab
    'max_val_samples': 1000,     # Reduced for Colab
    'k_sparse': 64,              # Reduced from 128
    'training': {
        'batch_size': 16,        # Reduced from 32
        'epochs': 5,             # Reduced from 10
        'learning_rate': 1e-3,
        'weight_decay': 1e-4,
        'warmup_steps': 500,
        'max_grad_norm': 1.0,
        'scheduler_type': 'cosine'
    }
}

# Save config
os.makedirs('experiments/01_sae_comparison', exist_ok=True)
with open('experiments/01_sae_comparison/colab_config.yaml', 'w') as f:
    yaml.dump(colab_config, f)

print("Config created!")


In [9]:
# Create a Colab-friendly config
import yaml

colab_config = {
    'seed': 42,
    'dataset': 'wikitext',
    'max_train_samples': 5000,   # Reduced for Colab
    'max_val_samples': 1000,     # Reduced for Colab
    'k_sparse': 64,              # Reduced from 128
    'training': {
        'batch_size': 16,        # Reduced from 32
        'epochs': 5,             # Reduced from 10
        'learning_rate': 1e-3,
        'weight_decay': 1e-4,
        'warmup_steps': 500,
        'max_grad_norm': 1.0,
        'scheduler_type': 'cosine'
    }
}

# Save config
os.makedirs('experiments/01_sae_comparison', exist_ok=True)
with open('experiments/01_sae_comparison/colab_config.yaml', 'w') as f:
    yaml.dump(colab_config, f)

print("Config created!")

Config created!


### 4. Run Quick Test

In [10]:
# %%
# Quick test to make sure everything works
!python experiments/01_sae_comparison/run_comparison.py \
    --model gpt2 \
    --layer 6 \
    --n_features 4096 \
    --batch_size 8 \
    --epochs 2 \
    --debug \
    --config experiments/01_sae_comparison/colab_config.yaml

# %% [markdown]
# ## 5. Run Main Experiment

# %%
# Run the full comparison (this will take 1-3 hours)
!python experiments/01_sae_comparison/run_comparison.py \
    --model gpt2 \
    --layer 6 \
    --n_features 8192 \
    --batch_size 16 \
    --epochs 5 \
    --config experiments/01_sae_comparison/colab_config.yaml

Traceback (most recent call last):
  File "/content/repo/experiments/01_sae_comparison/run_comparison.py", line 28, in <module>
    from src.models import KSparseSAE, GatedSAE, HybridSAE
ImportError: cannot import name 'KSparseSAE' from 'src.models' (unknown location)
Traceback (most recent call last):
  File "/content/repo/experiments/01_sae_comparison/run_comparison.py", line 16, in <module>
    import matplotlib.pyplot as plt
  File "/usr/local/lib/python3.11/dist-packages/matplotlib/pyplot.py", line 3774, in <module>
    @_copy_docstring_and_deprecators(Axes.pie)
     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/matplotlib/pyplot.py", line 196, in _copy_docstring_and_deprecators
    _add_pyplot_note(func, method)
  File "/usr/local/lib/python3.11/dist-packages/matplotlib/pyplot.py", line 254, in _add_pyplot_note
    doc = inspect.cleandoc(func.__doc__)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.11/inspect.py", line

### 6. Visualize Results

In [ ]:
# %%
# Load and display results
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import Image, display

# Find the latest results directory
results_dirs = sorted(Path('results').glob('sae_comparison_*'))
if results_dirs:
    latest_results = results_dirs[-1]
    print(f"Results directory: {latest_results}")

    # Load summary
    if (latest_results / 'summary_results.csv').exists():
        summary_df = pd.read_csv(latest_results / 'summary_results.csv')
        print("\nSummary Results:")
        display(summary_df)

    # Display plots
    for img_path in latest_results.glob('*.png'):
        print(f"\n{img_path.name}:")
        display(Image(str(img_path)))
else:
    print("No results found yet!")

### 7. Save to Google Drive (Optional)

# %%
# Mount Google Drive to save results
from google.colab import drive
drive.mount('/content/drive')

# %%
# Copy results to Drive
import shutil

if results_dirs:
    save_path = '/content/drive/MyDrive/sae_results'
    os.makedirs(save_path, exist_ok=True)
    
    # Copy latest results
    dest = os.path.join(save_path, latest_results.name)
    shutil.copytree(latest_results, dest, dirs_exist_ok=True)
    print(f"Results saved to: {dest}")

# %% [markdown]
# ## Alternative: Run Without Config File

# %%
# If config file approach doesn't work, run directly with all arguments
!python experiments/01_sae_comparison/run_comparison.py \
    --model gpt2 \
    --layer 6 \
    --n_features 8192 \
    --batch_size 16 \
    --epochs 5 \
    --learning_rate 1e-3 \
    --seed 42

### Memory Management (If Needed)

# %%
# If you run into memory issues, try these settings
!python experiments/01_sae_comparison/run_comparison.py \
    --model gpt2 \
    --layer 6 \
    --n_features 4096 \
    --batch_size 8 \
    --epochs 3 \
    --config experiments/01_sae_comparison/colab_config.yaml

# %%
# Clear GPU memory if needed
import torch
import gc

def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()
    
clear_memory()
print(f"GPU memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")